# AdaptiveMasterMSM: Prepare a simulation by using Gromacs
The aim of this notebook is to show that just calling class Controller we are able to do all the stuff done in the notebook called 'test_ala_functions.ipynb'

In [1]:
from adaptivemastermsm.system import system

In [2]:
system.check_gmx()

b'/usr/bin/gmx\n'


We invoke **Controller** class for doing, first, the preparation of the simulation box, and then, obtaining corresponding thermodynamic ensemble.

In [3]:
from adaptivemastermsm import controller

In [4]:
con = controller.Controller('data/alaTB.gro', forcefield='amber03', water='tip3p')

gmx pdb2gmx -f data/alaTB.gro -o data/alaTB.gro -p topol.top -ff amber03 -water tip3p
gmx editconf -f data/alaTB.gro -o conf_edit.gro -c -d 1.000000 -bt cubic
gmx solvate -cp conf_edit.gro -cs spc216.gro -p topol.top -o conf_solv.gro
gmx grompp -f data/minimization.mdp -c conf_solv.gro -p topol.top -o ions.tpr
gmx genion -s ions.tpr -o conf_solv_ions.gro -p topol.top -neutral
gmx grompp -c conf_solv_ions.gro -p topol.top -f data/mdp/min.mdp -o data/minim.tpr -r conf_solv_ions.gro
gmx mdrun -v -s data/minim.tpr -deffnm data/minim
gmx grompp -c data/minim.gro -p topol.top -f data/mdp/nvt.mdp -o data/posre.tpr -r data/minim.gro
gmx mdrun -v -s data/posre.tpr -deffnm data/posre
gmx grompp -c data/posre.gro -p topol.top -f data/mdp/npt.mdp -o data/npt.tpr -r data/posre.gro
gmx mdrun -v -s data/npt.tpr -deffnm data/npt


In [ ]:
con.adaptive_sampling(8, 10, sym=True, rate_mat=False, scoring='populations', n_epochs=2)


 Building MSM from 
 ['data/npt.xtc']
     # states: 32
 symmetrizing
MSM done
Runs for new epoch: [23  0  2 27  0 20 13  1]
Building entry for microstate 23
Building entry for microstate 0
Building entry for microstate 2
Building entry for microstate 27
Building entry for microstate 20
Building entry for microstate 13
Building entry for microstate 1
gmx pdb2gmx -f 23_18.gro -o 23_18.gro -p topol.top -ff amber03 -water tip3p
gmx editconf -f 23_18.gro -o conf_edit.gro -c -d 1.000000 -bt cubic
gmx solvate -cp conf_edit.gro -cs spc216.gro -p topol.top -o conf_solv.gro
gmx grompp -f data/minimization.mdp -c conf_solv.gro -p topol.top -o ions.tpr
gmx genion -s ions.tpr -o conf_solv_ions.gro -p topol.top -neutral
gmx grompp -c conf_solv_ions.gro -p topol.top -f data/mdp/min.mdp -o data/minim.tpr -r conf_solv_ions.gro
gmx mdrun -v -s data/minim.tpr -deffnm data/minim
gmx grompp -c data/minim.gro -p topol.top -f data/mdp/nvt.mdp -o data/posre.tpr -r data/minim.gro
gmx mdrun -v -s data/posre.t

 symmetrizing
MSM done
Runs for new epoch: [110  25  66 103 104 130 122 129]
Building entry for microstate 110
Building entry for microstate 25
Building entry for microstate 66
Building entry for microstate 103
Building entry for microstate 104
Building entry for microstate 130
Building entry for microstate 122
Building entry for microstate 129
gmx pdb2gmx -f 110_419.gro -o 110_419.gro -p topol.top -ff amber03 -water tip3p
gmx editconf -f 110_419.gro -o conf_edit.gro -c -d 1.000000 -bt cubic
gmx solvate -cp conf_edit.gro -cs spc216.gro -p topol.top -o conf_solv.gro
gmx grompp -f data/minimization.mdp -c conf_solv.gro -p topol.top -o ions.tpr
gmx genion -s ions.tpr -o conf_solv_ions.gro -p topol.top -neutral
gmx grompp -c conf_solv_ions.gro -p topol.top -f data/mdp/min.mdp -o data/minim.tpr -r conf_solv_ions.gro
gmx mdrun -v -s data/minim.tpr -deffnm data/minim
gmx grompp -c data/minim.gro -p topol.top -f data/mdp/nvt.mdp -o data/posre.tpr -r data/minim.gro
gmx mdrun -v -s data/posre.tp